In [1]:
import pandas as pd
import numpy as np

from sklearn.manifold import TSNE
from collections import Counter
import json

import random
import plotly.express as px

In [8]:
psm_counts_per_file = pd.read_csv("../data/SupplFig6_tSNEs/psm_counts_per_file.tsv", sep="\t")
mods = pd.read_csv("../data/Fig5_Modifications/Suppl_Table_2_Modification_Sites.tsv", sep="\t")

In [10]:
mods

,Modification,mod_pos_in_protein,protein_id,unimod_classification,Modified_Amino_Acid,unimod_id,mass_shift
0,Oxidation,122,P0ACF8,Artefact,M,35.0,15.994915
1,Oxidation,114,P0AAC0,Artefact,M,35.0,15.994915
2,Oxidation,68,P0A9P4,Artefact,M,35.0,15.994915
3,Oxidation,0,P0ABS1,Artefact,M,35.0,15.994915
4,Oxidation,229,P0A6B7,Artefact,M,35.0,15.994915
...,...,...,...,...,...,...,...
113253,dHex(1)Hex(3)HexA(2)HexNAc(2),433,P08997,O-linked glycosylation,S,1693.0,1390.439301
113254,Hex(3)HexNAc(3)NeuGc(1),272,P0ABK5,O-linked glycosylation,S,1696.0,1402.486920
113255,Hex(5)Phos(1),435,P0A9C5,O-linked glycosylation,T,1604.0,890.230448
113256,Hex(1)HexNAc(2)NeuAc(2),56,P0A7F6,O-linked glycosylation,S,1649.0,1150.402402


In [ ]:
ptms = mods[mods["unimod_classification"] == "Post-translational"]

observation_counts_per_ptm_per_file = ptms.groupby(
    ["Modification", "run"]
).size().reset_index(name="counts")

# to wide
observation_counts_per_ptm = (
    observation_counts_per_ptm_per_file.pivot(
        index="run", columns="Modification", values="counts"
    )
    .fillna(0)
    .reset_index()
)

# get metadata back
observation_counts_per_ptm = observation_counts_per_ptm.merge(
    psm_counts_per_file, on="run", how="left"
)

target_cols = ["characteristics[medium]", "characteristics[medium supplements]", "characteristics[aeration]", "characteristics[growth stage]", "characteristics[genetic background]", "characteristics[optical density]", "project_id", "characteristics[strain/breed]"]
metadata = observation_counts_per_ptm.merge(
    mods[["run"] + target_cols].drop_duplicates(),
    on="run",
    how="inner",
)

# replace pending with PRIDE Identifier
metadata["project_id"] = metadata["project_id"].replace(
    "Ecoli_PXD_pending", "PXD058808")

print(len(observation_counts_per_ptm))
print(len(metadata))
print(len(observation_counts_per_ptm_per_file))

# drop run column
observation_counts_per_ptm = observation_counts_per_ptm.drop(
    columns=["run"], axis=1
)

# normalise by psms
observation_counts_per_ptm = observation_counts_per_ptm.div(
    observation_counts_per_ptm["psms"], axis=0
)

# drop psms column
observation_counts_per_ptm = observation_counts_per_ptm.drop(
    columns=["psms"], axis=1
)

x = observation_counts_per_ptm.values

KeyError: 'run'

In [24]:
x

array([[0.00000000e+00, 0.00000000e+00, 3.04785126e-04, ...,
        2.03190084e-04, 5.07975211e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.32558140e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.50878073e-04, ...,
        5.01756147e-05, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.67723281e-05, 0.00000000e+00, 8.03169844e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.61465251e-05, 2.61465251e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [25]:
observation_counts_per_ptm

,3-phosphoglyceryl,AFB1_Dialdehyde,Acetyl,Acetyldeoxyhypusine,Acetylhypusine,Amidino,Ammonia-loss,Biotin,Bromo,CHDH,...,cGMP,cGMP+RMP-loss,glyoxalAGE,hydroxyisobutyryl,maleimide3,maleimide5,phosphoRibosyl,pupylation,pyrophospho,serotonylation
0,0.000000,0.000000,0.000305,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.000203,0.00000,0.0,0.0,0.000203,0.000051,0.0
1,0.000000,0.000000,0.000233,0.000047,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.000047,0.00000,0.0,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000251,0.000000,0.0,0.0,0.0,0.000050,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.0,0.000050,0.000000,0.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000019,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00002,0.000000,0.00002,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0
559,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000079,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0
560,0.000027,0.000000,0.000080,0.000000,0.0,0.0,0.0,0.000054,0.0,0.0,...,0.0,0.000027,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0
561,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000025,0.0,0.0,...,0.0,0.000025,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0


In [26]:
perplexity = np.arange(5, 55, 5)
divergence = []

for i in perplexity:
    model = TSNE(n_components=2, init="pca", perplexity=i)
    reduced = model.fit_transform(x)
    divergence.append(model.kl_divergence_)
fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

In [28]:
tsne = TSNE(n_components=2, random_state=0, perplexity=40)
projections = tsne.fit_transform(x)

principalDf = pd.DataFrame(data = projections, columns = ['TSNE_1', 'TSNE_2'])
finalDf = pd.concat([principalDf, metadata[target_cols]], axis=1)

fig = px.scatter(
    projections, x=0, y=1,
    color=finalDf['project_id'], labels={'color': 'project_id'},
    color_discrete_sequence=px.colors.qualitative.Alphabet,
    width=1200,
    height=1000
)
fig.show()

In [12]:
#target_cols = ["characteristics[medium]", "characteristics[medium supplements]", "characteristics[aeration]", "characteristics[growth stage]", "characteristics[genetic background]", "characteristics[optical density]", "project_id", "characteristics[strain/breed]"]

for factor_value in target_cols:
    fig = px.scatter(
        projections, x=0, y=1,
        color=finalDf[factor_value], labels={'color': factor_value},
        color_discrete_sequence=px.colors.qualitative.Alphabet,
        width=1200,
        height=1000
    )
    fig.show()